# Chapter 38: Building a DeFi Yield Aggregator

---

Yield aggregators are one of the most popular and powerful applications in DeFi. They automatically move user funds between different lending protocols and liquidity pools to maximize returns, saving users time and gas costs. In this chapter, we'll build a simplified yield aggregator vault inspired by Yearn Finance. Our vault will accept deposits of a stablecoin (e.g., DAI), and automatically allocate funds to the highest-yielding strategy among Aave and Compound. We'll implement the ERC-4626 standard for tokenized vaults, create reusable strategy contracts, and handle compounding, fees, and emergency withdrawals. By the end, you'll understand the core mechanics of yield aggregators and be able to build your own.

---

## 38.1 Project Overview

### 38.1.1 Requirements and Features

Our yield aggregator will have the following features:

- **Deposits**: Users deposit a specific asset (e.g., DAI) into the vault and receive vault shares (yVault tokens) representing their proportional ownership.
- **Withdrawals**: Users can redeem their shares for the underlying asset, plus any accrued yield.
- **Strategies**: The vault manager can allocate funds to multiple yield-generating strategies (e.g., lending on Aave, lending on Compound). Strategies are smart contracts that implement a standard interface.
- **Harvesting**: The vault periodically "harvests" yield from strategies, reinvests it, and updates the share price.
- **Fees**: The protocol takes a small performance fee (e.g., 2% of profits) and a management fee (e.g., 0.5% annually).
- **Emergency pause**: The vault can be paused in case of emergency, preventing deposits and withdrawals.

We'll also implement a simple mechanism to choose the best strategy based on current APY (though in practice, this would be more complex and likely involve off-chain oracles).

### 38.1.2 Architecture Design

Our system consists of:

- **Vault Contract**: ERC-4626 compliant vault that holds user funds, issues shares, and interacts with strategies.
- **Strategy Interface**: Standard interface that all strategies must implement (deposit, withdraw, harvest, balanceOf, etc.).
- **Aave Strategy**: A concrete strategy that deposits funds into Aave's lending pool (aToken).
- **Compound Strategy**: A concrete strategy that deposits funds into Compound (cToken).

```
┌─────────────────────────────────────────────────────────────┐
│                         Vault                               │
│  • ERC-4626 shares                                          │
│  • deposit()                                                │
│  • withdraw()                                               │
│  • allocate() / rebalance()                                 │
│  • harvest()                                                │
└─────────────────────────────────────────────────────────────┘
                              │
            ┌─────────────────┼─────────────────┐
            │                 │                 │
            ▼                 ▼                 ▼
    ┌───────────────┐ ┌───────────────┐ ┌───────────────┐
    │ Strategy Aave │ │Strategy Compound││  ... other    │
    │  • deposit()  │ │  • deposit()  │ │  strategies   │
    │  • withdraw() │ │  • withdraw() │ └───────────────┘
    │  • harvest()  │ │  • harvest()  │
    └───────────────┘ └───────────────┘
```

**Data Flow:**
1. User deposits DAI into Vault → Vault mints shares.
2. Vault holds DAI in its balance.
3. Vault manager (or keeper) calls `allocate()` to move funds to the best strategy.
4. Strategy deposits DAI into Aave/Compound, receiving interest-bearing tokens (aDAI/cDAI).
5. Over time, interest accrues.
6. Keeper calls `harvest()` to claim interest, convert back to DAI, and reinvest.
7. Fees are taken from profits and added to treasury.

---

## 38.2 Smart Contract Development

We'll use OpenZeppelin contracts for ERC-20, ERC-4626, and access control. We'll also interact with Aave and Compound protocols.

### 38.2.1 ERC-4626 Vault Standard

ERC-4626 is a standard for tokenized vaults that simplifies integration. It defines functions like `deposit`, `mint`, `withdraw`, `redeem`, `totalAssets`, and `convertToShares`. Our vault will inherit from OpenZeppelin's `ERC4626`.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC20/extensions/ERC4626.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
import "@openzeppelin/contracts/security/Pausable.sol";
import "@openzeppelin/contracts/security/ReentrancyGuard.sol";
import "./interfaces/IStrategy.sol";

contract YieldVault is ERC4626, Ownable, Pausable, ReentrancyGuard {
    // List of strategies
    IStrategy[] public strategies;
    mapping(IStrategy => bool) public isStrategy;

    // Fees: performance fee (e.g., 20% = 2000 basis points)
    uint256 public constant PERFORMANCE_FEE_BPS = 2000; // 20%
    uint256 public constant MANAGEMENT_FEE_BPS = 50; // 0.5% annual (simplified)
    uint256 public constant FEE_DENOMINATOR = 10000;

    address public treasury;

    // Last harvest timestamp for management fee calculation
    uint256 public lastHarvest;

    // Events
    event StrategyAdded(IStrategy indexed strategy);
    event StrategyRemoved(IStrategy indexed strategy);
    event Harvested(uint256 profit, uint256 performanceFee, uint256 managementFee);

    constructor(
        IERC20 asset_,
        string memory name_,
        string memory symbol_,
        address treasury_
    ) ERC4626(asset_, name_, symbol_) Ownable() {
        treasury = treasury_;
        lastHarvest = block.timestamp;
    }

    // --- Strategy Management (onlyOwner) ---
    function addStrategy(IStrategy strategy) external onlyOwner {
        require(!isStrategy[strategy], "Already added");
        strategies.push(strategy);
        isStrategy[strategy] = true;
        emit StrategyAdded(strategy);
    }

    function removeStrategy(IStrategy strategy) external onlyOwner {
        require(isStrategy[strategy], "Not a strategy");
        // Remove from array (simple swap and pop)
        for (uint i = 0; i < strategies.length; i++) {
            if (address(strategies[i]) == address(strategy)) {
                strategies[i] = strategies[strategies.length - 1];
                strategies.pop();
                break;
            }
        }
        isStrategy[strategy] = false;
        emit StrategyRemoved(strategy);
    }

    // --- Allocation Logic ---
    // This is a simplified "best strategy" selection.
    // In production, you'd use a more sophisticated system (e.g., off-chain keeper with APY data).
    function allocate() external onlyOwner {
        uint256 totalBalance = totalAssets();
        if (totalBalance == 0) return;

        // Withdraw from all strategies first? Actually, we want to rebalance.
        // For simplicity, we'll just deposit into the strategy with highest APY.
        // But first, we need to know current balances.
        IStrategy best = _getBestStrategy();
        if (address(best) == address(0)) return;

        // Withdraw from all other strategies and deposit into best
        for (uint i = 0; i < strategies.length; i++) {
            IStrategy strat = strategies[i];
            uint256 stratBalance = strat.balanceOf();
            if (stratBalance > 0) {
                strat.withdraw(stratBalance);
            }
        }

        // Deposit all into best strategy
        uint256 available = asset.balanceOf(address(this));
        if (available > 0) {
            best.deposit(available);
        }
    }

    function _getBestStrategy() internal view returns (IStrategy) {
        // Simple placeholder: return first strategy with highest APY? 
        // APY is not on-chain; in practice, you'd use an oracle or keeper.
        // For this example, we'll just return the first strategy if any.
        if (strategies.length > 0) {
            return strategies[0];
        }
        return IStrategy(address(0));
    }

    // --- Harvest ---
    // Collect profits from all strategies, reinvest after fees.
    function harvest() external nonReentrant whenNotPaused {
        uint256 totalProfit = 0;

        for (uint i = 0; i < strategies.length; i++) {
            IStrategy strat = strategies[i];
            uint256 before = asset.balanceOf(address(this));
            strat.harvest();
            uint256 after = asset.balanceOf(address(this));
            if (after > before) {
                totalProfit += after - before;
            }
        }

        // Calculate fees
        uint256 performanceFee = (totalProfit * PERFORMANCE_FEE_BPS) / FEE_DENOMINATOR;
        uint256 managementFee = _calculateManagementFee();
        uint256 totalFees = performanceFee + managementFee;

        if (totalFees > 0) {
            asset.transfer(treasury, totalFees);
        }

        // Reinvest remaining profits (if any)
        uint256 reinvestAmount = asset.balanceOf(address(this));
        if (reinvestAmount > 0) {
            IStrategy best = _getBestStrategy();
            if (address(best) != address(0)) {
                best.deposit(reinvestAmount);
            }
        }

        lastHarvest = block.timestamp;
        emit Harvested(totalProfit, performanceFee, managementFee);
    }

    function _calculateManagementFee() internal view returns (uint256) {
        uint256 timePassed = block.timestamp - lastHarvest;
        uint256 annualFee = (totalAssets() * MANAGEMENT_FEE_BPS) / FEE_DENOMINATOR;
        return (annualFee * timePassed) / 365 days;
    }

    // --- Overrides for ERC-4626 ---
    function totalAssets() public view override returns (uint256) {
        uint256 total = asset.balanceOf(address(this));
        for (uint i = 0; i < strategies.length; i++) {
            total += strategies[i].balanceOf();
        }
        return total;
    }

    // Before withdrawing, ensure we have enough assets (pull from strategies if needed)
    function _withdraw(address caller, address receiver, address owner, uint256 assets, uint256 shares)
        internal
        override
        nonReentrant
        whenNotPaused
        returns (uint256)
    {
        uint256 available = asset.balanceOf(address(this));
        if (available < assets) {
            uint256 missing = assets - available;
            // Withdraw from strategies proportionally? For simplicity, withdraw from first strategy.
            for (uint i = 0; i < strategies.length; i++) {
                if (missing == 0) break;
                uint256 stratBalance = strategies[i].balanceOf();
                if (stratBalance > 0) {
                    uint256 toWithdraw = stratBalance < missing ? stratBalance : missing;
                    strategies[i].withdraw(toWithdraw);
                    missing -= toWithdraw;
                }
            }
            require(missing == 0, "Insufficient liquidity");
        }
        return super._withdraw(caller, receiver, owner, assets, shares);
    }

    // Pause functionality
    function pause() external onlyOwner {
        _pause();
    }

    function unpause() external onlyOwner {
        _unpause();
    }
}
```

### 38.2.2 Strategy Interface

We define a common interface that all strategies must implement.

```solidity
// interfaces/IStrategy.sol
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC20/IERC20.sol";

interface IStrategy {
    // Returns the total amount of underlying assets managed by this strategy.
    function balanceOf() external view returns (uint256);

    // Deposits underlying assets into the strategy.
    function deposit(uint256 amount) external;

    // Withdraws underlying assets from the strategy.
    function withdraw(uint256 amount) external;

    // Harvests profits, converting any earned interest back to underlying asset.
    function harvest() external;

    // Returns the underlying asset token.
    function asset() external view returns (IERC20);
}
```

### 38.2.3 Aave Strategy

Aave v3 lending pool allows users to deposit assets and receive aTokens (interest-bearing). We'll interact with the pool to deposit and withdraw.

```solidity
// strategies/AaveStrategy.sol
pragma solidity ^0.8.19;

import "./interfaces/IStrategy.sol";
import "@openzeppelin/contracts/token/ERC20/IERC20.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
import "@openzeppelin/contracts/security/ReentrancyGuard.sol";

interface IAavePool {
    function supply(address asset, uint256 amount, address onBehalfOf, uint16 referralCode) external;
    function withdraw(address asset, uint256 amount, address to) external returns (uint256);
    function getReserveData(address asset) external view returns (uint256, uint128, uint128, uint128, uint128, uint128, uint40, address);
}

contract AaveStrategy is IStrategy, Ownable, ReentrancyGuard {
    IERC20 public immutable underlying;
    IAavePool public immutable pool;
    address public immutable aToken; // The interest-bearing token (e.g., aDAI)

    constructor(IERC20 _underlying, IAavePool _pool, address _aToken) {
        underlying = _underlying;
        pool = _pool;
        aToken = _aToken;
        // Approve pool to spend underlying
        underlying.approve(address(pool), type(uint256).max);
    }

    function asset() external view override returns (IERC20) {
        return underlying;
    }

    function balanceOf() public view override returns (uint256) {
        // Our balance in the pool is the aToken balance (which represents underlying + interest)
        // But for underlying amount, we need to know exchange rate. Simplified: aToken balance * exchange rate.
        // For Aave, aTokens are pegged 1:1 initially, but accrue interest over time.
        // So aToken balance roughly equals underlying + interest.
        return IERC20(aToken).balanceOf(address(this));
    }

    function deposit(uint256 amount) external override nonReentrant onlyOwner {
        require(amount > 0, "Amount must be > 0");
        // Transfer underlying from vault to this strategy
        underlying.transferFrom(msg.sender, address(this), amount);
        // Supply to Aave
        pool.supply(address(underlying), amount, address(this), 0);
    }

    function withdraw(uint256 amount) external override nonReentrant onlyOwner {
        require(amount > 0, "Amount must be > 0");
        // Withdraw from Aave
        uint256 withdrawn = pool.withdraw(address(underlying), amount, address(this));
        // Transfer underlying back to vault
        underlying.transfer(msg.sender, withdrawn);
    }

    function harvest() external override nonReentrant onlyOwner {
        // For Aave, interest accrues automatically; no need to claim.
        // But we may need to realize gains? In practice, we just keep aTokens.
        // The vault's totalAssets includes aToken balance, so it's accounted for.
        // We could optionally claim COMP rewards if using Aave v2, but v3 has no rewards.
        // For simplicity, we do nothing.
    }
}
```

### 38.2.4 Compound Strategy

Compound works similarly, using cTokens.

```solidity
// strategies/CompoundStrategy.sol
pragma solidity ^0.8.19;

import "./interfaces/IStrategy.sol";
import "@openzeppelin/contracts/token/ERC20/IERC20.sol";
import "@openzeppelin/contracts/access/Ownable.sol";

interface IComptroller {
    function enterMarkets(bytes calldata cTokens) external returns (uint[] memory);
}

interface ICToken {
    function mint(uint256 mintAmount) external returns (uint256);
    function redeemUnderlying(uint256 redeemAmount) external returns (uint256);
    function balanceOf(address owner) external view returns (uint256);
    function underlying() external view returns (address);
}

contract CompoundStrategy is IStrategy, Ownable {
    IERC20 public immutable underlying;
    ICToken public immutable cToken;
    IComptroller public immutable comptroller;

    constructor(IERC20 _underlying, ICToken _cToken, IComptroller _comptroller) {
        underlying = _underlying;
        cToken = _cToken;
        comptroller = _comptroller;
        underlying.approve(address(cToken), type(uint256).max);
        // Enter market (to enable collateral, but not needed for simple lending)
        address[] memory markets = new address[](1);
        markets[0] = address(cToken);
        comptroller.enterMarkets(markets);
    }

    function asset() external view override returns (IERC20) {
        return underlying;
    }

    function balanceOf() public view override returns (uint256) {
        // cToken.balanceOf() returns cTokens; we need underlying amount.
        // Exchange rate can be queried, but for simplicity we return cToken balance.
        // In production, you'd use cToken.exchangeRateCurrent().
        return cToken.balanceOf(address(this)) * 1; // placeholder, need exchange rate
    }

    function deposit(uint256 amount) external override onlyOwner {
        require(amount > 0, "Amount must be > 0");
        underlying.transferFrom(msg.sender, address(this), amount);
        require(cToken.mint(amount) == 0, "Mint failed");
    }

    function withdraw(uint256 amount) external override onlyOwner {
        require(amount > 0, "Amount must be > 0");
        require(cToken.redeemUnderlying(amount) == 0, "Redeem failed");
        underlying.transfer(msg.sender, amount);
    }

    function harvest() external override onlyOwner {
        // Compound does not auto-compound; interest is reflected in cToken exchange rate.
        // No action needed.
    }
}
```

### 38.2.5 Fees and Compounding Logic

Our vault calculates performance fee on profits and management fee based on time. The `harvest` function collects fees and reinvests remaining profits. In practice, management fee might be taken from total assets periodically, but we simplified.

---

## 38.3 Testing with Mainnet Fork

We'll test our vault by forking mainnet to interact with real Aave and Compound contracts.

**Hardhat test example:**
```javascript
const { expect } = require("chai");
const { ethers } = require("hardhat");

describe("YieldVault", function () {
  let vault, aaveStrategy, compoundStrategy;
  let dai, aDai, cDai;
  let owner, user, treasury;
  let aavePool, comptroller;

  beforeEach(async function () {
    [owner, user, treasury] = await ethers.getSigners();

    // Fork mainnet at a specific block
    await network.provider.request({
      method: "hardhat_reset",
      params: [{
        forking: {
          jsonRpcUrl: `https://eth-mainnet.alchemyapi.io/v2/${process.env.ALCHEMY_KEY}`,
          blockNumber: 15000000,
        },
      }],
    });

    // Get mainnet addresses
    dai = await ethers.getContractAt("IERC20", "0x6B175474E89094C44Da98b954EedeAC495271d0F");
    aDai = await ethers.getContractAt("IERC20", "0x028171bCA77440897B824Ca71D1c56caC55b68A3"); // Aave v2 aDAI
    cDai = await ethers.getContractAt("ICToken", "0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643"); // cDAI
    aavePool = await ethers.getContractAt("IAavePool", "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"); // Aave v2 lending pool
    comptroller = await ethers.getContractAt("IComptroller", "0x3d9819210A31b4961b30EF54bE2aeD79B9c9Cd3B"); // Compound comptroller

    // Deploy strategies
    const AaveStrategy = await ethers.getContractFactory("AaveStrategy");
    aaveStrategy = await AaveStrategy.deploy(dai.address, aavePool.address, aDai.address);
    const CompoundStrategy = await ethers.getContractFactory("CompoundStrategy");
    compoundStrategy = await CompoundStrategy.deploy(dai.address, cDai.address, comptroller.address);

    // Deploy vault
    const YieldVault = await ethers.getContractFactory("YieldVault");
    vault = await YieldVault.deploy(dai.address, "Yield DAI", "yDAI", treasury.address);

    // Add strategies to vault
    await vault.addStrategy(aaveStrategy.address);
    await vault.addStrategy(compoundStrategy.address);

    // Transfer some DAI to user for testing
    const whale = "0x..." // DAI whale address
    await network.provider.request({
      method: "hardhat_impersonateAccount",
      params: [whale],
    });
    const whaleSigner = await ethers.getSigner(whale);
    await dai.connect(whaleSigner).transfer(user.address, ethers.parseEther("10000"));
  });

  it("Should deposit and allocate", async function () {
    // User deposits DAI into vault
    const depositAmount = ethers.parseEther("1000");
    await dai.connect(user).approve(vault.address, depositAmount);
    await vault.connect(user).deposit(depositAmount, user.address);

    // Check vault total assets
    expect(await vault.totalAssets()).to.equal(depositAmount);

    // Allocate to strategies (owner only)
    await vault.connect(owner).allocate();

    // Check that strategies have funds
    const aaveBalance = await aaveStrategy.balanceOf();
    const compoundBalance = await compoundStrategy.balanceOf();
    // Since our best strategy logic just picks first, all funds should go to Aave
    expect(aaveBalance).to.be.closeTo(depositAmount, ethers.parseEther("1")); // slight rounding
    expect(compoundBalance).to.equal(0);
  });

  it("Should harvest and take fees", async function () {
    // Deposit and allocate
    const depositAmount = ethers.parseEther("1000");
    await dai.connect(user).approve(vault.address, depositAmount);
    await vault.connect(user).deposit(depositAmount, user.address);
    await vault.connect(owner).allocate();

    // Simulate time passing (interest accrues)
    await ethers.provider.send("evm_increaseTime", [7 * 24 * 60 * 60]); // 1 week
    await ethers.provider.send("evm_mine");

    // Harvest
    await vault.connect(owner).harvest();

    // Check that fees were taken
    const treasuryBalance = await dai.balanceOf(treasury.address);
    expect(treasuryBalance).to.be.gt(0);
  });
});
```

---

## 38.4 Security Considerations

- **Reentrancy**: All state-changing functions use `nonReentrant`.
- **Access control**: Only owner can add/remove strategies, allocate, harvest.
- **Pausable**: Owner can pause deposits/withdrawals in case of emergency.
- **Strategy risk**: Strategies could be hacked or become insolvent. The vault should have emergency withdrawal to remove funds from a strategy.
- **Oracle risk**: APY decisions may rely on off-chain data; if incorrect, funds could be allocated poorly. Use decentralized oracles or keepers.
- **Slippage**: Withdrawals may need to pull from strategies; ensure enough liquidity.

---

## 38.5 Frontend Integration

A simple frontend could allow users to:
- Deposit/withdraw via vault.
- View their share balance and estimated APY.
- (Admin) trigger harvest and allocation.

We can use ethers.js and the vault's ABI.

---

## 38.6 Extensions and Future Work

- **More strategies**: Add Yearn, Convex, etc.
- **Dynamic allocation**: Use a keeper network (Chainlink Keepers) to automate harvest and rebalance.
- **Vaults with multiple assets**.
- **Gas optimization**: Batch operations.
- **Emergency shutdown**: Allow users to withdraw even if a strategy fails.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 38 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  We built a yield aggregator vault that:                       │
│    • Accepts deposits and issues shares (ERC-4626)             │
│    • Allocates funds to multiple lending strategies            │
│    • Harvests yield and reinvests after fees                   │
│    • Supports Aave and Compound strategies                     │
│                                                                 │
│  Key concepts:                                                 │
│    • ERC-4626 standard for tokenized vaults                    │
│    • Strategy interface for pluggable yield sources            │
│    • Performance and management fees                           │
│    • Mainnet fork testing with real protocols                  │
│                                                                 │
│  Security: reentrancy guard, access control, pausable          │
│  Testing: Hardhat with forked mainnet                          │
│                                                                 │
│  This vault can be extended with more strategies, keeper       │
│  automation, and better allocation logic.                      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 39 – Account Abstraction (ERC-4337). We'll explore how smart contract wallets enable gasless transactions, social recovery, and improved UX, with hands-on implementation using ERC-4337.